In [ ]:
# default_exp core

# Lockdown dates

> Retrieve the dates that countries around the world went into lockdown during the covid pandemic.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sean")

'Hello Sean!'

In [ ]:
assert say_hello("Sean")=="Hello Sean!"